In [101]:
import pandas as pd
import numpy as np
import random
import requests

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

!pip install folium
import folium

from bs4 import BeautifulSoup

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


<H1> Use beautifulSoup to extract<H1>

In [102]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url)
soup = BeautifulSoup(html.content, "html5lib")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})    


df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [103]:
df.shape

(103, 3)

<H1> merge latitude and longitude informatino into existing dataframe <H1>

In [104]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

Coord = pd.read_csv(url)
Coord.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [105]:
df_merge = df.merge(Coord, left_on='PostalCode', right_on='Postal Code')
df_merge

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


In [106]:
df_merge.shape

(103, 6)

In [107]:
address = "Toronto, ON, Canada"
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude

df_merge_reduced = df_merge[df_merge['Borough'].str.contains('Toronto')]

map_toronto = folium.Map(location=[lat, lon], zoom_start=10)
for lat, lon, borough, neighborhood in zip(df_merge_reduced['Latitude'], df_merge_reduced['Longitude'], df_merge_reduced['Borough'], df_merge_reduced['Neighborhood']):
        label= '{}, {}'.format(neighborhood, borough)
        label=folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3, parse_html=True).add_to(map_toronto)
        
map_toronto

<H1> Use KMeans to cluster based on longitude and latitude informatino <H1>

In [108]:
KM_df = df_merge_reduced[['Latitude','Longitude']]

from sklearn.cluster import KMeans
KM = KMeans(init = "k-means++", n_clusters = 5, n_init = 12)
KM.fit(KM_df)
KM.labels_


array([0, 0, 0, 2, 0, 0, 4, 0, 1, 2, 0, 4, 2, 0, 4, 2, 0, 2, 3, 3, 3, 3,
       1, 3, 4, 1, 3, 4, 1, 3, 4, 3, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [109]:
df_merge_reduced['Labels'] = KM.labels_
df_merge_reduced.head()


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Labels
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,0
15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,0
19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,2
20,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,0


In [110]:
cluster_map = folium.Map(location=[lat, lon], zoom_start=10)

# color
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers
markers_colors = []
for lat, lon, nei, cluster in zip(df_merge_reduced['Latitude'], df_merge_reduced['Longitude'], df_merge_reduced['Neighborhood'], df_merge_reduced['Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
    
cluster_map